In [ ]:
import cirq
from qramcircuits.toffoli_decomposition import ToffoliDecompType, ToffoliDecomposition
from mathematics.draper0406142 import CarryLookaheadAdder
import qramcircuits.bucket_brigade as bb
from utils.counting_utils import *
import optimizers as qopt

In [ ]:
def main(n_qubits, decomp_scenario):
        
    #Create the qubits of the circuits
    
    qubits = []
    for i in range(n_qubits):
        qubits.append(cirq.NamedQubit("a" + str(i)))

    #Create the search
    #search = [0, 1, 2, 3]
    #search = list(range(0, 2**n_qubits-1))
    
    print("********* Bucket Brigade *********")

    bbcircuit = bb.BucketBrigade(qubits, decomp_scenario = decomp_scenario)
    
    print(bbcircuit.circuit.to_text_diagram(use_unicode_characters=True, qubit_order = bbcircuit.qubit_order))

    # #Verification
    print("Verify nqubits:      {}\n".format(bbcircuit.verify_number_qubits()))
    print("Verify Depth:        {}\n".format(bbcircuit.verify_depth(
        Alexandru_scenario=decomp_scenario.parallel_toffolis))
    )
    print("Verify T_count:      {}\n".format(bbcircuit.verify_T_count()))
    print("Verify T_depth:      {}\n".format(bbcircuit.verify_T_depth(
        Alexandru_scenario=decomp_scenario.parallel_toffolis))
    )
    print("Verify H_count:      {}\n".format(bbcircuit.verify_hadamard_count(
        Alexandru_scenario=decomp_scenario.parallel_toffolis))
    )
    # print("Verify CNOT_c:   {}\n".format(bbcircuit.verify_cnot_count(
    #     Alexandru_scenario=olivia_decomposition.parallel_toffolis))
    # )

    # qopt.CommuteTGatesToStart().optimize_circuit(bbcircuit.circuit)
    #
    #print(bbcircuit.circuit)

    # qopt.SearchCNOTPattern().optimize_circuit(bbcircuit.circuit)

    # qopt.CancelNghCNOTs().apply_until_nothing_changes(bbcircuit.circuit,
    #                                                   cu.count_cnot_of_circuit)

    return bbcircuit

In [ ]:
n_qubits = 2
decomp_scenario = bb.BucketBrigadeDecompType(
    [
        ToffoliDecompType.ZERO_ANCILLA_TDEPTH_4_COMPUTE,    # fan_in_decomp
        ToffoliDecompType.ZERO_ANCILLA_TDEPTH_4,  # mem_decomp
        ToffoliDecompType.ZERO_ANCILLA_TDEPTH_0_UNCOMPUTE,    # fan_out_decomp
    ],
    True
)

In [ ]:
bbcircuit = main(n_qubits = n_qubits, decomp_scenario=decomp_scenario)

In [ ]:
from utils.counting_utils import *
old_moments = bbcircuit.circuit.moments
old_num_T_gates = count_num_gates(bbcircuit.circuit)

In [ ]:
new_circuitbbcircuit.remove_T_gates(percentage=0.2, inplace=False)

In [ ]:
new_moments = bbcircuit.circuit.moments
new_num_T_gates = count_t_of_circuit(bbcircuit.circuit)

In [ ]:
bbcircuit.circuit.to_text_diagram(use_unicode_characters=True, qubit_order = bbcircuit.qubit_order)

In [ ]:
import itertools
lists = list(itertools.product([0, 1], repeat=2))
l = []
for item in lists:
    string = ''
    for i in item:
        string = string + str(i)
    print(string)
    l.append(string)

## Check input/output

In [ ]:
import itertools

def create_binary_strings(n):
    bin_array = list(itertools.product([0, 1], repeat=n))
    bin_strings = []
    for item in bin_array:
        string = ''
        for i in item:
            string = string + str(i)
        bin_strings.append(string)
    return bin_strings

In [ ]:
from tests.test_BB_remove_T import *

for n_qubits in range(2, 13):
    decomp_scenario = choose_decomposition(decomp_ID='1')
    bbcircuit = create_BB_circuit(n_qubits=n_qubits, decomp_scenario=decomp_scenario)
    for initial_state in create_binary_strings(n_qubits):
        bcircuit, bbcircuit_modified, freq_origin, freq_mod = test_remove_T(bbcircuit, initial_state=initial_state, percentage=0.2, inplace=True, repetitions=1000)
    if n_qubits==3: break
    #verify_counts(bbcircuit, bbcircuit_modified, decomp_scenario)

In [ ]:
qubits = []
for i in range(n_qubits):
    qubits.append(cirq.NamedQubit("a" + str(i)))

In [ ]:
# original one
bbcircuit = bb.BucketBrigade(qubits, decomp_scenario = decomp_scenario)

In [ ]:
#a0,a1 -> Memory cell
#  0    0  -> 00
#  0    1  -> 01
#  1    0  -> 10
#  1    1  -> 11

In [ ]:
import numpy as np
np.array(bbcircuit.all_qubits()).shape

In [ ]:
bbcircuit.all_qubits()[-1].name

In [ ]:
bbcircuit.circuit.append(cirq.measure(bbcircuit.all_qubits()[-1], key='target'))
bbcircuit_2.circuit.append(cirq.measure(bbcircuit_2.all_qubits()[-1], key='target'))

In [ ]:
def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

In [ ]:
# for original circuit
simulator = cirq.Simulator()
result = simulator.run(bbcircuit.circuit, repetitions=100)
print("Results:")
frequencies = result.histogram(key='target', fold_func=bitstring)
print(f'Sampled results:\n{frequencies}')

In [ ]:
# for modified circuit
simulator = cirq.Simulator()
result = simulator.run(bbcircuit_2.circuit, repetitions=100)
print("Results:")
frequencies = result.histogram(key='target', fold_func=bitstring)
print(f'Sampled results:\n{frequencies}')

In [ ]:
from cirqpyzx import CirqPyZX

circ_optimized = CirqPyZX(bbcircuit)
circ_optimized = circ_optimized.optimize()

In [ ]:
import pyzx as zx

zx.draw(circ_optimized)

In [ ]:
from cirq.contrib.qasm_import.qasm import circuit_from_qasm
circ_optimized_qasm = circ_optimized.to_qasm()
circ_optimized_cirq = circuit_from_qasm(circ_optimized_qasm)

In [ ]:
circ_optimized.stats_dict(depth=True)

In [ ]:
circ_optimized_cirq

In [ ]:
print(circ_optimized_cirq.to_text_diagram(use_unicode_characters=True))

In [ ]:
from utils.counting_utils import *

count_rzT_depth_of_circuit(circ_optimized_cirq)

In [ ]:
count_rzT_of_circuit(circ_optimized_cirq)

In [ ]:
circ_optimized_cirq.ops.

In [ ]:
print("Verify T_d:      {}\n".format(circ_optimized_cirq.verify_T_depth(
        Alexandru_scenario=decomp_scenario.parallel_toffolis))
    )